# Optimization of Noise Generator

We want to find the right Hyperparameters for the Noise Generator.
For the Optimization, we will use the `Optuna` Framework

The correct Hyperparameters for the Noise will be found for:
- `GeFeU`
- `GEMU`
- `Gradient Ascent`

In [ ]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import random

import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.join('..', 'src'))

import gefeu
import gemu
import mlp_dataclass
import metrics
from helper import load_models_dict

___
## GeFeU

In [ ]:
import logging
import sys
import pickle
import optuna
import os

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study1_name = "GenOptiGeFeU"  # Unique identifier of the study.
storage1_name = "sqlite:///{}.db".format("HP_Opti")

if os.path.exists("sampler_gefeu.pkl"):
    restored1_sampler = pickle.load(open("sampler_gefeu.pkl", "rb"))
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True, sampler=restored1_sampler)
else:
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True)

In [ ]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    n_layers = trial.suggest_int('n_layers', 1, 10)

    Layers = [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]
    Layers = Layers[:n_layers]

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]
    
    unlearned = gefeu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gefeu.optimize(objective, n_trials=200)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gefeu.pkl", "wb") as fout:
    pickle.dump(study_gefeu.sampler, fout)

In [ ]:
u = load_models_dict(path="../data/models/mnist/all/test_ensemble")[1]
z = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[1]
valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
valid_dl = DataLoader(valid_ds, 56, shuffle=False)

In [ ]:
metrics.kl_divergence_between_models(z, u, valid_dl, device='cpu')

## Standard Values

In [ ]:
from src import gefeu

standard_model_gefeu = gefeu._main(
    model=u[0],
    dataset_name="mnist",
    t_Epochs=study_gefeu.best_params["opt_Epochs"],
    t_Batch_Size=study_gefeu.best_params["opt_Batch_Size"],
    t_Learning_Rate=study_gefeu.best_params["opt_Learning_Rate"],
    t_N2R_Ratio=study_gefeu.best_params["opt_N2R_Ratio"],
    t_Regularization_term=study_gefeu.best_params["opt_Regularization_term"],
    t_Layers=[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024][:study_gefeu.best_params["n_layers"]],
    t_Noise_Dim=study_gefeu.best_params["opt_Noise_Dim"],
    t_Impair_LR=study_gefeu.best_params["opt_Impair_LR"],
    t_Repair_LR=study_gefeu.best_params["opt_Repair_LR"],
    logs=True,
    model_eval_logs=False,
)

___
## GEMU

In [ ]:
import logging
import sys
import pickle
import optuna
import os


# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study2_name = "GenOptiGEMU"  # Unique identifier of the study.
storage2_name = "sqlite:///{}.db".format("HP_Opti")

if os.path.exists("sampler_gemu.pkl"):
    restored2_sampler = pickle.load(open("sampler_gemu.pkl", "rb"))
    study_gemu = optuna.create_study(study_name=study2_name, storage=storage2_name, load_if_exists=True, sampler=restored2_sampler)
else:
    study_gemu = optuna.create_study(study_name=study2_name, storage=storage2_name, load_if_exists=True)

In [ ]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    n_layers = trial.suggest_int('n_layers', 1, 10)

    Layers = [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]
    Layers = Layers[:n_layers]

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    unlearned = gemu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gemu.optimize(objective, n_trials=200)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gemu.pkl", "wb") as fout:
    pickle.dump(study_gemu.sampler, fout)

## Standard Values

In [ ]:
from src import gemu

standard_model_gemu = gemu._main(
    model=u[0],
    dataset_name="mnist",
    t_Epochs=study_gemu.best_params["opt_Epochs"],
    t_Batch_Size=study_gemu.best_params["opt_Batch_Size"],
    t_Learning_Rate=study_gemu.best_params["opt_Learning_Rate"],
    t_N2R_Ratio=study_gemu.best_params["opt_N2R_Ratio"],
    t_Regularization_term=study_gemu.best_params["opt_Regularization_term"],
    t_Layers=[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024][:study_gemu.best_params["n_layers"]],
    t_Noise_Dim=study_gemu.best_params["opt_Noise_Dim"],
    t_Impair_LR=study_gemu.best_params["opt_Impair_LR"],
    t_Repair_LR=study_gemu.best_params["opt_Repair_LR"],
    logs=True,
    model_eval_logs=False,
)

___
## Gradient Ascent

In [ ]:
import logging
import sys
import pickle
import optuna
import os


# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study3_name = "OptiGA"  # Unique identifier of the study.
storage3_name = "sqlite:///{}.db".format("HP_Opti")

if os.path.exists("sampler_ga.pkl"):
    restored3_sampler = pickle.load(open("sampler_ga.pkl", "rb"))
    study_ga = optuna.create_study(study_name=study3_name, storage=storage3_name, load_if_exists=True, sampler=restored3_sampler)
else:
    study_ga = optuna.create_study(study_name=study3_name, storage=storage3_name, load_if_exists=True,)

In [ ]:
from unlearning import SimpleGradientAscent

def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0, 1)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 1, 256)

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    forget_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="only_erased",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )
    forget_dl = valid_dl = DataLoader(forget_ds, opt_Batch_Size, shuffle=False)

    unlearned = SimpleGradientAscent(
        model=train,
        unlearned_data=forget_dl,
        dataset_name="mnist",
        t_LR = opt_Learning_Rate,
        t_Epochs = opt_Epochs,
        ).unlearn()

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_ga.optimize(objective, n_trials=200)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_ga.pkl", "wb") as fout:
    pickle.dump(study_ga.sampler, fout)

## Standard Values

In [ ]:
forget_ds = mlp_dataclass.MNIST_CostumDataset(
    sample_mode="only_erased",
    train=True,
    test=False,
    balanced=False,
    dataset_name="mnist",
    download=False,
)
forget_dl = DataLoader(forget_ds, batch_size=study_ga.best_params["opt_Batch_Size"], shuffle=False)

unlearned_model_ga = SimpleGradientAscent(
    model=u[0],
    unlearned_data=forget_dl,
    dataset_name="mnist",
    t_LR = study_ga.best_params["opt_Learning_Rate"],
    t_Epochs = study_ga.best_params["opt_Epochs"],
    ).unlearn()

___